# Chat with PDF page images

This notebook demonstrates how to convert PDF pages to images and send them to a vision model for inference

## Authenticate to GitHub Models


In [15]:
import os

import openai
from dotenv import load_dotenv

load_dotenv(".env", override=True)

openai_client = openai.OpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com")

## Convert PDFs to images

In [16]:
%pip install Pillow PyMuPDF

Note: you may need to restart the kernel to use updated packages.


In [17]:
import pymupdf
from PIL import Image

filename = "plants.pdf"
doc = pymupdf.open(filename)
for i in range(doc.page_count):
    page = doc.load_page(i)
    pix = page.get_pixmap()
    original_img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    original_img.save(f"page_{i}.png")


## Send images to vision model

In [18]:
import base64


def open_image_as_base64(filename):
    with open(filename, "rb") as image_file:
        image_data = image_file.read()
    image_base64 = base64.b64encode(image_data).decode("utf-8")
    return f"data:image/png;base64,{image_base64}"


response = openai_client.chat.completions.create(
    model="Llama-4-Maverick-17B-128E-Instruct-FP8",
    temperature=0.2,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "text": "Output the table on this page as Markdown table",
                    "type": "text",
                },
                {
                    "image_url": {"url": open_image_as_base64("page_0.png")},
                    "type": "image_url",
                },
            ],
        }
    ]
)

print(response.choices[0].message.content)

| Species | Common Name | Qty | Size | Price | Source County | Notes |
| --- | --- | --- | --- | --- | --- | --- |
| Annual |  |  |  |  |  |  |
| Centromadia pungens | Common tarweed | 8 | 4.5S | $1.83 | Unknown | 75% off sale |
| Epilobium densiflorum | Dense Spike-primrose | 3 | 4.5S | $3.65 | San Mateo | 50% off sale |
| Eschscholzia caespitosa | Tufted Poppy | 119 | D-16S | $3.60 | Unknown | 50% off sale |
| Eschscholzia californica | California poppy | 85 | D-16S | $3.60 | Bay Area | 50% off sale |
| Eschscholzia californica 'Purple Gleam' | Purple Gleam Poppy | 2 | D-16S | $3.60 | Unknown | 50% off sale |
| Eschscholzia californica var. maritima | Coastal California Poppy | 137 | D-16S | $3.60 | Unknown | 50% off sale |
| Madia elegans | Tarweed | 6 | 4.5S | $1.83 | Unknown | 75% off sale |
| Mentzelia lindleyi | Lindley's Blazing Star | 35 | 3.5S | $3.65 | Unknown | 50% off sale |
| Symphyotrichum subulatum | Slim marsh aster | 10 | D-16S | $5.40 | Contra Costa | 25% off sale |
